# 📅 Day 2 — Data Cleaning & Preparation

*RTU Data Analysis & Visualization CPD course*

**📚 Instruction (3h)**  
- 🧹 Handling missing values  
- 🗑 Removing duplicates  
- 🔄 Data type conversion  
- 📅 Parsing dates  
- 🏗 Feature engineering basics  
- 🔗 Combining datasets  
- 🏷 Intro to categorical encoding  

**🛠 Practical (1h)**  
- 🧽 Clean a messy dataset  
- 🔀 Merge with a secondary dataset  

**🔄 Reflection (1h)**  
- 🧐 Review: common pitfalls in cleaning  
- 💬 Discuss real-world cleaning challenges  
- 📝 Recap exercise: identify cleaning steps for a small example dataset

## 🎯 Goals for the Day
- Strengthen Python basics (functions, loops, if/else, file handling)
- Learn to process raw messy text files into usable form
- Apply pandas methods to clean incomplete/messy data
- Merge multiple datasets into a single unified dataframe

## 💡 Motivation / Explanation

### Introduction to Data Cleaning and Preparation

- **Why cleaning is critical before analysis**
  - Raw data is almost never ready for direct analysis
  - Errors, inconsistencies, and missing information can distort results
  - Proper cleaning ensures reliability, reproducibility, and trust in analysis outcomes

- **Real-world examples of messy data**
  - 🧹 Handling missing values - Weather records with missing timestamps or corrupt values
  - 🗳️ Survey responses with inconsistent categories (e.g., "Male", "male", "M")
  - 🗑 Removing duplicates - Financial transactions with duplicate entries
  - 🗑 Log files with noise lines, system messages, or broken encodings
  - 📅 Parsing dates - Event logs with inconsistent timestamp formats
  - 🔄 Data type conversion - User age recorded as text instead of numbers

> Think of data cleaning as *“washing vegetables before cooking”* — not exciting, but essential for a good meal.


## Weather Dataset: `latvia_meteo_1925_messy.zip`

Let us imagine we are helping Toms Bricis with weather data analysis for the year 1925. We have come across a bundle of messy text files that require cleaning and preparation.

- **What it is:** a bundle of **five “messy” text files** (≈50 rows each) simulating daily measurements from Latvian stations in **1925**:
  - **Rīga-University** — *Period 1 (Jan–Mar)*
  - **Rīga-University** — *Period 2 (Sep–Nov)*
  - **Liepāja** — *Apr–Jun*
  - **Mērsrags** — *Feb–May*
  - **Alūksne** — *Oct–Dec*

- **Columns present (but order varies by file):**  
  `date`, `t_max_c`, `t_min_c`, `precip_24h_mm`, `precip_type`, `present_weather_code`, `notes`

- **Deliberate “messiness” to practice cleaning:**
  - **Different separators:** `;`, `,`, `|`, and **TAB** (documented in each file’s `# fields=` header).
  - **Mixed column order** across files (use the header to map columns).
  - **Date formats vary** (`YYYY-MM-DD`, `DD.MM.YYYY`, `YYYY/MM/DD`, `DD-MM-YYYY`, `MM-DD-YYYY`, `YYYY.MM.DD`) and sometimes **include a time** (e.g., `07:00`).
  - **Numeric quirks:** decimal **commas** (e.g., `0,6`), **units** in strings (e.g., `0.8 mm`), and the Latvian word **“nulle”** for zero.
  - **Missing values** sprinkled in as `""`, `NA`, `—`, `-999`.
  - **Codes as strings** with possible leading zeros (e.g., `present_weather_code = "05"`).
  - **Free-text `notes`** in Latvian from the station master (may be blank/missing).

- **Intended skills to practice (Day 2):**
  - Detect & use **separators/column order** from headers.
  - **Parse heterogeneous dates** (with optional times).
  - Normalize **numerics/units** (decimal commas, `mm`, worded zeros).
  - Unify **missing values** and enforce **types** (e.g., cast weather codes to integers).
  - Keep useful **categorical text** (`precip_type`, `notes`) intact.

As part of our workflow we will want to verify whether the above descriptions of messiness hold true for our specific dataset files. This will help us tailor our cleaning approach effectively.


## Part 1: Python Fundamentals for Data Cleaning

For our first part we will use basic Python programming skills to explore and understand the dataset structure before diving into the cleaning process.

### 🔑 Key Idea - Loops Go Brrr

One of key advantages of programming is that we can automate repetitive tasks using loops. This is especially useful when working with datasets, as it allows us to apply the same operations to multiple rows or files without having to write redundant code.

Similarly loops let us figure out an approach that works for a single file and then easily adapt it to others.


### Getting ready for work

Typically in a finished notebook (and also normal scripts / programs), we want to start with a clear setup phase. This includes:

1. **Importing Libraries:** Load all necessary libraries at the beginning.
2. **Setting Up Paths:** Define file paths and other constants.
3. **Configuring Options:** Set any options or preferences (e.g., display settings).

By organizing our code this way, we make it easier to understand and modify later on.


In [1]:
# usually we start with general Python imports
from pathlib import Path # for file and file path related tasks
import sys, platform, os, io, shutil, zipfile, re # system related tasks
from datetime import datetime # this is because datetime module has datetime class
# first datetime
print(f"Today : {datetime.now().isoformat(timespec='seconds')}")
# now Python version
print(f"Python : {sys.version}")

# then we import external libraries
# external - not part of Python installation
# on Google Colab those are already installed
try:
    import pandas as pd
    print('pandas:', pd.__version__)
except ImportError:
    print(f"pandas not installed. Install with `pip install pandas`.")
    # for excel support extra instructions
    print(f"Install `openpyxl` for Excel support with `pip install openpyxl`.")
# requests is a widely used network library that makes internet "requests" easier
try:
    import requests
except ImportError:
    requests = None
    print('requests not installed. Install with `pip install requests`.')


# we can also print out what type of environment we are running in, this could show OS information
print('Runtime:', platform.platform())
# We could show system RAM and free RAM but that would require either a non standard library
# or we would have to write some extra functions we skip this for now
# you can ask LLM to write these functions for you
# alternatively there are external libraries like psutil that do this out of the box

# Let us show our current drive space
print(f"Total Current Drive Space: {shutil.disk_usage('/').total / (1024**3):.2f} GB")
print(f"Free Current Drive Space: {shutil.disk_usage('/').free / (1024**3):.2f} GB")

# Current Working Directory
print(f"Current Working Directory: {Path.cwd()}") # more important in local computer
# in Google Colab this should give you /content
# note that in some cases you might not want to provide all this information to the public, if you have a super secret computer...


Today : 2025-08-21T15:17:56
Python : 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
pandas: 2.2.2
Runtime: Linux-6.1.123+-x86_64-with-glibc2.35
Total Current Drive Space: 107.72 GB
Free Current Drive Space: 68.88 GB
Current Working Directory: /content


### 🧑‍💻 Functions

### What is the idea behind functions in programming?

In programming, a function is a block of reusable code that performs a specific task. Think of it like a miniature program within your main program. Functions are designed to:

- **Break down complex problems:** Large problems can be divided into smaller, manageable parts, each handled by a function. This makes code easier to write, understand, and debug.
- **Avoid repetition (DRY principle - Don't Repeat Yourself):** If you need to perform the same set of actions multiple times, you can define a function once and call it whenever needed, rather than writing the same code repeatedly.
- **Improve code organization and readability:** Functions group related code together, making the overall structure of your program clearer and easier to follow.
- **Enhance code reusability:** Once a function is defined, it can be used in different parts of the same program or even in other programs.
- **Simplify debugging:** If there's an issue, you can isolate the problem to a specific function, making it easier to find and fix the error.

In essence, functions are tools for modularity and abstraction in programming, allowing you to create more organized, efficient, and maintainable code.

### What is a function in Python?

In Python, a function is defined using the `def` keyword, followed by the function name, parentheses `()`, and a colon `:`. The code block within the function is indented. Functions can optionally take inputs called *arguments* (placed inside the parentheses) and can return a value using the `return` keyword.

Here's a basic structure of a Python function:

In [2]:
def greet(name: str = 'student') -> str:
    """
    Returns a friendly greeting for the given name.
    If no name is provided, defaults to 'student'.
    """
    # name is a variable local to this function
    # Use an f-string to insert the name into the greeting
    # note we are not printing the greeting here, just returning it for use by another part of the code
    return f"Hello, {name}!" # we can insert pretty much any type of data in f-strings

# also nothing should be happening besides the function being added to our memory
# we have not called the function yet

In [3]:
# Call the function and print the result
greeting = greet() # assign results of greet() function to greeting variable
print(greeting)

Hello, student!


In [5]:
# Now that I have this function I can make other greetings
my_greeting = greet("Valdis")
print(my_greeting)
numeric_greeting = greet(80800) # function expects str, but no penalty in this case
print(numeric_greeting)

Hello, Valdis!
Hello, 80800!


### Type Hints

Type hints in Python provide a way to indicate the expected data types of variables, function parameters, and return values. They help improve code readability and enable better static analysis by tools like linters and IDEs.

However, they have no actual power at runtime and are not enforced by the Python interpreter. They serve as a guideline for developers and can be checked using static type checkers like mypy.

I liken them to "documentation for your code." Just as documentation helps users understand how to use your code, type hints help developers understand what types of values are expected.



In [6]:
# let's see one more example with simple add function
def add(a: int, b: int) -> int:
    return a + b

# so this function expects only integers and returns integer

# however it will work with any values that support adding



In [7]:
print(add(2,2))
print(add(3.14, 2.71))
print(add("Valdis", " RTU"))

# so again type hints are useful (AI will happily make them for us)
# but in Python they only serve as a guiding light

4
5.85
Valdis RTU


In [11]:
# let's make a multiplication function that takes two integers and returns an integer
def multiply(a: int, b: str = "*") -> int:
    return a * b

print(multiply(2,3))
print(multiply(3.1415926, 44444))
print(multiply("Beer ", 5)) # funnily Python lets us multiply Strings with integers
print("*"*80)
print(multiply(20))

6
139624.9415144
Beer Beer Beer Beer Beer 
********************************************************************************
********************


### Function to download and unzip file from url

Below is a more complicated function that does two things: it downloads a zip file from a given URL and then extracts its contents to a specified directory.

Theoretically speaking it would be better to have two functions one that downloads the file and another that extracts it.

In general functions should do one thing and do it well.

When you need to do more than one thing you can combine them into a single function, but be mindful of keeping the function focused and not overly complex.


In [13]:
text = "A quick brown fox jumped over a sleeping dog"
words = text.split() # split creates a list of strings from string
# by default it splits by any whitespace
print(words)
comma_text = "Valdis, Līga, Maija, Rūta, Ede"
comma_words = comma_text.split(",")
print(comma_words)

['A', 'quick', 'brown', 'fox', 'jumped', 'over', 'a', 'sleeping', 'dog']
['Valdis', ' Līga', ' Maija', ' Rūta', ' Ede']


In [14]:
def download_and_unzip(url: str, target_folder: str | Path = 'sample_data') -> Path:
    """
    Downloads a ZIP file from the given URL and extracts it to the target folder.
    Returns the path to the folder where files were extracted.
    """
    target = Path(target_folder)  # Make sure target is a Path object
    target.mkdir(parents=True, exist_ok=True)  # Create the folder if it doesn't exist
    filename = url.split('/')[-1]  # Get the file name from the URL
    # in case of URL the file name is the last one (so index -1 means last one in a list)
    # next we create full path where we will save the zip file
    zip_path = target / filename  # Full path to save the ZIP file
    # check if library exists
    if requests is None:
        raise RuntimeError('requests required.')
    # Download the file in chunks (good for large files)
    with requests.get(url, stream=True, timeout=60) as r:
        r.raise_for_status()  # Raise an error if download failed
        with open(zip_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    # Unzip the downloaded file
    with zipfile.ZipFile(zip_path, 'r') as zf:
        zf.extractall(target) # this command given a zip file as target will unzip ALL files
    # Optionally, remove the ZIP file after extraction
    # zip_path.unlink(missing_ok=True)  # Remove the ZIP file
    return target  # Return the folder where files were extracted


**Practice dataset for Part 1:** `latvia_meteo_1925_messy.zip` (5 text files)

- URL: https://github.com/ValRCS/RTU_Data_Analysis_Visualization_CPD/raw/refs/heads/main/data/latvia_meteo_1925_messy.zip

In [15]:
# let's download the messy files zip
url = "https://github.com/ValRCS/RTU_Data_Analysis_Visualization_CPD/raw/refs/heads/main/data/latvia_meteo_1925_messy.zip"
print(f"Will download and unzip from following url: {url}")
# let's download the file and extract it under day_2_data
download_and_unzip(url, Path("day_2_data"))

Will download and unzip from following url: https://github.com/ValRCS/RTU_Data_Analysis_Visualization_CPD/raw/refs/heads/main/data/latvia_meteo_1925_messy.zip


PosixPath('day_2_data')

In [16]:
text_files = sorted(Path("day_2_data").glob("*.txt"))
for p in text_files:
    print(p)

day_2_data/aluksne_1925.txt
day_2_data/liepaja_1925.txt
day_2_data/mersrags_1925.txt
day_2_data/riga_university_1925_p1.txt
day_2_data/riga_university_1925_p2.txt


In [17]:
# let's add Google Colab specific code that will offer download to your computer of all the files extracted
# this will not work locally since you already have these files locally!
from google.colab import files # this is Google Colab specific
# if you were local you would already have these files locally
for p in sorted(Path("day_2_data").glob("*.txt")): # glob looks in current folder
    files.download(p)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 📂 File Handling

First let me show you how to read whole text file into one big text string

In [18]:
# let's make a function that takes a file Path or string and optional encoding with default utf-8 and returns text string
def get_file_contents(path: Path | str, encoding: str = 'utf-8') -> str:
    """
    Reads the contents of a text file and returns it as a string.
    """
    # so path could be string or Path
    # file could have any extension but it should contain text of some sort
    with open(path, mode='r', encoding=encoding) as f:
        # file stream is open here under f variable which has type IOStream(sic)
        content = f.read() # so we came up with name content for our inner variable
    # here file is automatically closed after the with block
    return content

In [21]:
# let's read aluksne into memory
# aluksne = get_file_contents(Path("day_2_data/aluksne_1925.txt"))
aluksne = get_file_contents("day_2_data/aluksne_1925.txt")
print(aluksne[:300]) # let's print first 300 characters

# station_name=Alūksne
# period=(Oct–Dec)
# separator_hint=|
# columns_in_this_file= notes | present_weather_code | t_max_c | precip_24h_mm | date | precip_type | t_min_c
# note: some values intentionally messy (units, words, missing, time in date)
# fields=notes|present_weather_code|t_max_c|precip_


In [ ]:
# print last 200 characters
print(aluksne[-200:]) # Python offers two types of indexing
# positive from 0 to len(iterable)-1
# negative from -len(iterable) to -1

![Python indexing](https://developers.google.com/static/edu/python/images/hello.png)

### Iterating over file line at a time

Above example showed how we could read a whole file into memory.
However that means we would be working with file as one big string as a whole. We could do some replace operations.

Much more often we will want to work on file one line(row) at a time.

### 🔄 For Loops

For loops in Python let us iterate over a sequence (like a list, tuple, or string, or other iterables such as lines in a file) and perform an action for each item in that sequence.

General syntax of for loops is:

```Python
for element in <iterable>:
    <action>
    <more optional action>
```

Note the indentation, as usual in Python after : we have indentation to indicate the block of code that belongs to the for loop.

In [ ]:
def count_lines(path: Path | str) -> int:
    """
    Counts the total number of lines and the number of non-empty lines in a file.
    Returns a tuple: (total_lines, nonempty_lines)
    """
    total = 0
    with open(path, 'r', encoding='utf-8', errors='replace') as f:
        for line in f:
            total += 1
            # we could do something more here with the line but here we just count the contents

    return total # Return the line count

In [ ]:
# let's count lines in aluksne_1925
count_lines(Path("day_2_data/aluksne_1925.txt"))

42

## Determining the separator character

Our files have helpfully provided a separator hint without which it would be very hard to determine.

`# separator_hint=|`
`# separator_hint=TAB`
`# separator_hint=,`

Let's write a function that extracts hint from the file


In [ ]:
def get_sep(path: Path) -> str:
    """
    We look for first line that contains
    `# separator_hint=|`
`# separator_hint=TAB`
`# separator_hint=,`
    """
    needle = "separator_hint="
    sep = None # we start with assumption that we do not know the separator
    # lets open file and go through line by line
    with open(path, 'r', encoding='utf-8', errors='replace') as f:
        for line in f:
            if needle in line: # we check for presence of needle
            # we could have used regular expression but no need here
                # let's find whatever is after needle
                # so we split by needle at take last part
                sep = line.split(needle)[-1].strip()
    # if sep is TAB we need to return \t
    if sep == "TAB": # special case TAB which we need to convert
        return "\t"
    return sep

In [ ]:
# let's see if we can find separator for aluksne_1925
get_sep(Path("day_2_data/aluksne_1925.txt"))

'|'

In [ ]:
# let's find it for all txt files that do not contain good or bad
for p in sorted(Path("day_2_data").glob("*.txt")):
    if "good" in p.name or "bad" in p.name: continue
    # we could have used bad as well
    print(p, get_sep(p))

day_2_data\aluksne_1925.txt |
day_2_data\liepaja_1925.txt ,
day_2_data\mersrags_1925.txt 	
day_2_data\riga_university_1925_p1.txt ;
day_2_data\riga_university_1925_p2.txt ;


### 📥 Loading Cleaned Files into DataFrames

In [ ]:
# Let us write a function that will take a file path
# This function will go through file line by line
# If line with separator_hint= is found we save in sep value the last part of this line after stripping whitespace
# any other line that starts with hash (#) is ignored
# all other lines are split using sep value and stored as a list of lists (2d)
# once all lines are read the 2d list is converted to Dataframe and returned
def load_messy_file(path: Path | str) -> pd.DataFrame:
    sep = None
    lines = [] # empty list
    # with guarantees closing of file at the end
    with open(path, 'r', encoding='utf-8', errors='replace') as f:
        # we loop through file one row at a time
        for line in f:
            if line.startswith('# separator_hint='):
                # extract separator
                sep = line.split('=')[-1].strip()
                # add special case TAB -> \t
                if sep == "TAB":
                    sep = "\t"
            if line.startswith("#"): # order is important we check this AFTER hint check
                continue # we go to next line
            if sep:
                # we split the row by separator and add it to our list of lines
                lines.append(line.split(sep))
    # Here file is closed - that's good and safe!
    # all that remains is to create a dataframe from our lines
    df = pd.DataFrame(lines) # we leave the challenge of creating appropriate names for later
    return df

In [ ]:
# okay let's write a function that takes src folder containing text files
# function takes also dst folder for output of xlsx files that will be obtained by saving Dataframes from text files
def create_xlsx_files(src: Path | str, dst: Path | str):
    # first read text files in src
    src = Path(src) # to make sure that src is actually Path
    text_files = sorted(src.glob('*.txt'))

    # create dst if it does not exist
    dst = Path(dst)
    dst.mkdir(exist_ok=True)
    # now simply loop through text_files and create Dataframe and save in dst as text file with xlsx suffix
    for p in text_files:
        df = load_messy_file(p)
        df.to_excel(dst / (p.stem + ".xlsx"))

### Saving converted files into XLSX

Now that we have needed functions we can use them to convert text files to XLSX format.

Of course the text files should match the specific format (have separator hint and naturally have lines of content separated by that separtor

In [ ]:
# let'' take day_2_data text files
# and convert them to xlsx
# let's use new folder for that
# let's call it day_2_xlsx
INPUT_DIR = "day_2_data"
OUTPUT_DIR = "day_2_xlsx"

create_xlsx_files(INPUT_DIR, OUTPUT_DIR)

## Part 2: Guided Exercise — Latvia Weather Data (Extra Messy)

**Duration:** ~30 minutes  
**Dataset:** `latvia_meteo_1925_extra_messy.zip`  
**URL:** https://github.com/ValRCS/RTU_Data_Analysis_Visualization_CPD/raw/refs/heads/main/data/latvia_meteo_1925_extra_messy.zip

### 🎯 Objective
Convert multiple extra-messy weather text files   into XLSX files into a new folder of your choice

### Hint

Simply run the provided functions with folders of your choice.

### Advanced Users

You can start working on extracting column names from the original files. We will do that after break.

In [ ]:
# --- SKELETON (students fill in) ---
EXTRA_URL = 'https://github.com/ValRCS/RTU_Data_Analysis_Visualization_CPD/raw/refs/heads/main/data/latvia_meteo_1925_extra_messy.zip'
DATA_DIR = Path('day_2_exercise')

# 1) Download & unzip
# download_and_unzip(EXTRA_URL, DATA_DIR)

# # 2) Print contents of DATA DIR

# data_files = sorted(DATA_DIR.glob('*.txt'))
# for file_name in data_files: # file_names is just a variable name
#   print(file_name)

# 2) Inspect: list files & counts
# for p in sorted(DATA_DIR.glob('*.txt')):
#     print(p.name, '->', count_lines(p))

# 3) Clean all files
# results = clean_files(DATA_DIR)
# results

# 4) Load cleaned files
# dfs_extra = {}
# for p in sorted(DATA_DIR.glob('*.good.txt')):
#     dfs_extra[p.stem] = load_cleaned_file(p)
# {k: v.head() for k, v in dfs_extra.items()}

day_2_exercise/dobele_1925.txt
day_2_exercise/mersrags_extra_1925.txt
day_2_exercise/pavilosta_1925.txt


In [ ]:
# # let us test on day_2_exercise folder
# INPUT_FOLDER = Path("day_2_exercise") # this should actually exist
# OUTPUT_FOLDER = Path("day_2_exercise_output") # this can be any valid folder name
# create_xlsx_files(INPUT_FOLDER, OUTPUT_FOLDER)

### 🧪 Checkpoints
- At least **N≥3** cleaned files successfully load into DataFrames.
- No parsing exceptions on `.head()` or `.info()`.
- You can explain (in comments) which rules your `is_good_line` used.

### 🛠 Extension (Optional)
- Write a variant `clean_files(folder, out_dir=Path('data/cleaned'))` that writes outputs into a subfolder.
- Add a **regex-based** `is_good_line_regex` that only keeps lines starting with `YYYY-MM-DD`.

## Part 2b: Extracting column names automatically

We were able to extract the data from the files and load them into DataFrames and then save them into XLSX files.

One negative is that we do not have column names just numbers.

Thus we would like to extract column names automatically

## Part 3: Pandas-Specific Data Cleaning

Now we will focus on cleaning the data using pandas.



### Overview
In this section, you will standardize each DataFrame from Part 2 so they share a **common schema** and are ready to merge.

### Target Schema (example)
- `date` (datetime)
- `station` (string/category)
- `t_min` (float)
- `t_max` (float)
- `precip` (float)

### Typical Operations
1. **Column detection & renaming** – bring different column names to a shared set
2. **Type coercion** – numbers via `pd.to_numeric(errors='coerce')`, dates via `pd.to_datetime(errors='coerce')`
3. **Missing values** – `dropna` or `fillna` depending on context
4. **Duplicates** – `.duplicated()` + `.drop_duplicates()`
5. **Categoricals** – normalize text (`strip`, `title`, `upper`) and `astype('category')` if useful
6. **Validation** – quick assertions (e.g., date not null, temperature ranges plausible)

### Step-by-Step Guide
1) **Pick one DataFrame** from `dfs_extra` and print `.head()`, `.columns`, `.info()`
2) **Map columns** to target names (e.g., `temp_min` → `t_min`)
3) **Coerce**:
   - `date = pd.to_datetime(df['date'], errors='coerce')`
   - `df[['t_min','t_max','precip']] = df[['t_min','t_max','precip']].apply(pd.to_numeric, errors='coerce')`
4) **Handle missing**: start conservative (e.g., drop rows missing `date` or all temperature columns)
5) **Standardize station names**: `df['station'] = df['station'].astype(str).str.strip().str.title()`
6) **Check duplicates** and remove
7) **Repeat** for all DataFrames

### Common Pitfalls & Tips
- Treat ambiguous `-` or `NA` strings as missing (`na_values=["-","NA","N/A"]` if you re-read with `read_csv`)
- Some files might have **merged columns**; split using `.str.split(',', expand=True)` when necessary
- If a file lacks a column, create it with `pd.NA` so the schema lines up later

### 🧱 Skeleton: Inspect & Rename

In [ ]:
# Example skeleton for one dataframe named df
# df = dfs_extra['some_file']
# print(df.head()); print(df.columns); df.info()

# rename_map = {
#     'Date': 'date', 'DATE':'date',
#     'Station':'station', 'City':'station',
#     'Tmin':'t_min', 'TminC':'t_min', 'Min':'t_min',
#     'Tmax':'t_max', 'TmaxC':'t_max', 'Max':'t_max',
#     'Precip':'precip', 'Rain':'precip'
# }
# df = df.rename(columns=lambda c: rename_map.get(str(c), str(c).strip().lower()))


### 🧱 Skeleton: Type Coercion & Missing Handling

In [ ]:
# required_cols = ['date','station','t_min','t_max','precip']
# for c in required_cols:
#     if c not in df.columns:
#         df[c] = pd.NA

# df['date'] = pd.to_datetime(df['date'], errors='coerce')
# for c in ['t_min','t_max','precip']:
#     df[c] = pd.to_numeric(df[c], errors='coerce')

# # Drop rows with no usable date
# df = df.dropna(subset=['date'])

# # Optional: fill precip missing with 0 if domain-appropriate
# # df['precip'] = df['precip'].fillna(0)

### 🧱 Skeleton: Text Normalization & Duplicates

In [ ]:
# df['station'] = df['station'].astype(str).str.strip().str.title()
# before = len(df)
# df = df.drop_duplicates()
# print('Removed', before - len(df), 'duplicate rows')

### 🧪 Suggested Sanity Checks

In [ ]:
# assert df['date'].notna().all(), 'Null dates remain'
# # Optional plausibility checks (adjust to real units)
# assert (df['t_min'] <= df['t_max']).dropna().all(), 'Found t_min > t_max'

## Part 4: Merging Cleaned DataFrames

### Goal
Combine all standardized DataFrames into **one big DataFrame** with a **unified column structure**.

### Strategy
1. **Define the target schema** used in Part 3.
2. **Align each DataFrame** to the schema (add missing columns, reorder).
3. **Concatenate** with `pd.concat`.
4. **Final cleanup**: deduplicate, reindex, and sort by date/station.
5. **Save outputs** (`CSV` or `Parquet`) for Day 3 (EDA).

### Integration Checklist
- All DataFrames have columns: `date, station, t_min, t_max, precip`
- Dtypes are consistent across DataFrames
- No catastrophic loss of rows during coercion
- Final row count equals the sum of inputs minus duplicates

### 🧱 Skeleton: Alignment & Concatenation

In [ ]:
# Suppose you have a dict of cleaned dfs: dfs_clean
# target_cols = ['date','station','t_min','t_max','precip']

# def coerce_to_schema(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
#     for c in cols:
#         if c not in df.columns:
#             df[c] = pd.NA
#     # Reorder and drop extras for now
#     return df[cols]

# aligned = [coerce_to_schema(d.copy(), target_cols) for d in dfs_clean.values()]
# big = pd.concat(aligned, axis=0, ignore_index=True)
# big = big.drop_duplicates().reset_index(drop=True)
# big = big.sort_values(['date','station'])
# big.head()

### 🧾 Export for Day 3

In [ ]:
# out_dir = Path('outputs'); out_dir.mkdir(exist_ok=True)
# big.to_csv(out_dir / 'latvia_meteo_1925_cleaned_merged.csv', index=False)
# # Optional: Parquet for speed/size
# # big.to_parquet(out_dir / 'latvia_meteo_1925_cleaned_merged.parquet', index=False)

## 🔄 Reflection
- What kinds of messiness were easier to fix with **Python basics**?
- What kinds of messiness required **pandas**?
- What are the risks of “over-cleaning” or discarding too much data?